# 4) Dictionaries — Exercises

**Learning goals:** CRUD, iteration patterns, safe access (`get`, `setdefault`), merging, grouping, transforming, nested access.

### Warm-ups

1. **Make profile with defaults**

```python
def make_profile(name, email=None, country="IN"):
    ...
p = make_profile("A","a@example.com")
assert p["name"]=="A" and p["country"]=="IN"
```

2. **Safe get path** — nested dict path or default

```python
def get_in(d, path, default=None):
    ...
cfg = {"db":{"host":"localhost","port":5432}}
assert get_in(cfg, ["db","port"]) == 5432
assert get_in(cfg, ["db","user"], "postgres") == "postgres"
```

3. **Set with default container**

```python
def multimap_add(mm, key, value):
    """mm: dict key -> list; append value; create list if missing."""
    ...
mm = {}
multimap_add(mm,"a",1); multimap_add(mm,"a",2)
assert mm == {"a":[1,2]}
```

### Core

4. **Invert mapping (values unique)**
   If duplicate values appear, keep first key.

```python
def invert_unique(d):
    ...
assert invert_unique({"a":1,"b":2}) == {1:"a",2:"b"}
```

5. **Invert to sets (values may repeat)**

```python
def invert_to_sets(d):
    ...
assert invert_to_sets({"a":1,"b":1,"c":2}) == {1:{"a","b"}, 2:{"c"}}
```

6. **Merge with priority** — `override` wins

```python
def merge_priority(base, override):
    ...
assert merge_priority({"a":1,"b":2},{"b":9,"c":3}) == {"a":1,"b":9,"c":3}
```

7. **Group by key function**

```python
def group_by(xs, key_fn):
    ...
g = group_by(["apple","pear","plum","kiwi"], len)
assert g == {5:["apple"],4:["pear","plum","kiwi"]}
```

8. **Top-N by value**

```python
def top_n(d, n):
    ...
assert top_n({"a":3,"b":1,"c":5},2) == [("c",5),("a",3)]
```

9. **Join two “tables” by key** (left join)

```python
def left_join(left_rows, right_rows, key):
    """
    left_rows/right_rows: list of dicts
    Return: list of merged dicts; right fields may be missing.
    """
    ...
L = [{"id":1,"name":"A"},{"id":2,"name":"B"}]
R = [{"id":1,"score":90}]
out = left_join(L,R,"id")
assert out[0]["score"]==90 and out[1].get("score") is None
```

### Challenge

10. **Flatten nested dict with dot paths**

```python
def flatten_dot(d, parent_key=""):
    """
    {"a":{"b":1},"c":2} -> {"a.b":1, "c":2}
    """
    ...
assert flatten_dot({"a":{"b":1},"c":2}) == {"a.b":1,"c":2}
```


In [11]:
# 1) Make profile with defaults
def make_profile(name, email=None, country="IN"):
    return {"name": name, "email": email, "country": country}

p = make_profile("A","a@example.com")
assert p["name"]=="A" and p["country"]=="IN"

In [12]:
# 2) Safe get path — nested dict path or default
def get_in(d, path, default=None):
    cur = d
    for key in path:
        if not isinstance(cur, dict) or key not in cur:
            return default
        cur = cur[key]
    return cur

cfg = {"db":{"host":"localhost","port":5432}}
assert get_in(cfg, ["db","port"]) == 5432
assert get_in(cfg, ["db","user"], "postgres") == "postgres"

In [13]:
# 3) Set with default container
def multimap_add(mm, key, value):
    """mm: dict key -> list; append value; create list if missing."""
    mm.setdefault(key, []).append(value)

mm = {}
multimap_add(mm,"a",1); multimap_add(mm,"a",2)
assert mm == {"a":[1,2]}

In [14]:
# 4) Invert mapping (values unique); keep first key on duplicates
def invert_unique(d):
    out = {}
    for k, v in d.items():
        if v not in out:
            out[v] = k
    return out

assert invert_unique({"a":1,"b":2}) == {1:"a",2:"b"}

In [15]:
# 5) Invert to sets (values may repeat)
def invert_to_sets(d):
    out = {}
    for k, v in d.items():
        out.setdefault(v, set()).add(k)
    return out

assert invert_to_sets({"a":1,"b":1,"c":2}) == {1:{"a","b"}, 2:{"c"}}

In [16]:
# 6) Merge with priority — override wins
def merge_priority(base, override):
    out = dict(base)
    out.update(override)
    return out

assert merge_priority({"a":1,"b":2},{"b":9,"c":3}) == {"a":1,"b":9,"c":3}

In [17]:
# 7) Group by key function
def group_by(xs, key_fn):
    groups = {}
    for x in xs:
        k = key_fn(x)
        groups.setdefault(k, []).append(x)
    return groups

g = group_by(["apple","pear","plum","kiwi"], len)
assert g == {5:["apple"],4:["pear","plum","kiwi"]}

In [18]:
# 8) Top-N by value
def top_n(d, n):
    return sorted(d.items(), key=lambda kv: kv[1], reverse=True)[:max(0, n)]

assert top_n({"a":3,"b":1,"c":5},2) == [("c",5),("a",3)]

In [19]:
# 9) Join two “tables” by key (left join)
def left_join(left_rows, right_rows, key):
    index = {r[key]: r for r in right_rows}
    out = []
    for L in left_rows:
        R = index.get(L.get(key))
        merged = dict(L)            # copy left row
        if R is not None:
            for k, v in R.items():
                if k != key:
                    merged[k] = v
        else:
            # ensure right-side fields can be present as None? Not required;
            # assertion only checks .get("score") is None.
            pass
        out.append(merged)
    return out

L = [{"id":1,"name":"A"},{"id":2,"name":"B"}]
R = [{"id":1,"score":90}]
out = left_join(L,R,"id")
assert out[0]["score"]==90 and out[1].get("score") is None

In [20]:
# 10) Flatten nested dict with dot paths
def flatten_dot(d, parent_key=""):
    flat = {}
    for k, v in d.items():
        new_key = f"{parent_key}.{k}" if parent_key else str(k)
        if isinstance(v, dict):
            flat.update(flatten_dot(v, new_key))
        else:
            flat[new_key] = v
    return flat

assert flatten_dot({"a":{"b":1},"c":2}) == {"a.b":1,"c":2}